# Cubeviz base functionality with leaflet

1. Load a data cube
2. Scrub through the channels, displaying 2D and 1D
3. Median collapse cube
4. Stretch goal: Hover stats at cursor (x, y, counts, RA, Dec)

In [ ]:
import io
import logging
import random
import threading

import numpy as np

from astropy import visualization
from astropy.nddata import CCDData

import ipyleaflet
import ipywidgets

from IPython.display import display

import flask

%matplotlib inline
from matplotlib import pyplot as plt

Create sample data cube.

In [ ]:
# Not sure why we need this as array, but this is
# leftover from html_view_into_leaflet.ipynb
ccdis = []

In [ ]:
np.random.seed(1234)
cube = np.random.random(10000000).reshape((1000, 100, 100))

In [ ]:
ccdis.append(CCDData(cube, unit='count'))

Some functions copied over from `html_view_into_leaflet.ipynb`.

**TODO: These functions should be documented because I have no idea what they are supposed to do.**

In [ ]:
"""
Writing PNGs
"""

#-----------------------------------------------------------------------------
# Copyright (c) 2013, yt Development Team.
#
# Distributed under the terms of the Modified BSD License.
#
# The full license is in the file COPYING.txt, distributed with this software.
#-----------------------------------------------------------------------------

import matplotlib._png as _png

from io import BytesIO as StringIO


def call_png_write_png(buffer, width, height, filename, dpi):
    _png.write_png(buffer, filename, dpi)

    
def write_png(buffer, filename, dpi=100):
    width = buffer.shape[1]
    height = buffer.shape[0]
    call_png_write_png(buffer, width, height, filename, dpi)

    
def write_png_to_string(buffer, dpi=100, gray=0):
    width = buffer.shape[1]
    height = buffer.shape[0]
    fileobj = StringIO()
    call_png_write_png(buffer, width, height, fileobj, dpi)
    png_str = fileobj.getvalue()
    fileobj.close()
    return png_str

In [ ]:
# SERVER:

visdat = None
_last_stretch = None


# NOTE: Unlike html_view_into_leaflet.ipynb, there is extra
# arg named i_slice here for slicing the cube.
def re_stretch(stretch, i_slice):
    global visdat, _last_stretch
    visdat = np.flip(stretch(ccdis[0].data[i_slice]), 0)
    _last_stretch = stretch
    

def ccd_to_pngstr_app(dat):
    return write_png_to_string((dat*255).astype('uint8')[:,:,np.newaxis])


logstream = io.StringIO()
logging.basicConfig(stream=logstream)

app = flask.Flask(__name__)

   
# TODO: Something is not quite right, as this results in weird
# tiling behavior in the viewer.
@app.route('/fits<int:cachebuster>/<string:z>/<int:x>/<int:y>.png')
def get_subfits(z, x, y, cachebuster):
    # TODO: What is this trying to do?
    z = int(z)
    z0 = np.log2(256/visdat.shape[0])
    z = z - 1 + z0
    z = int(np.ceil(z))
    
    if z < 0:
        factor = 2**-z
        wid = 256 * factor
        xrng = slice(x*wid, (x+1)*wid, factor)
        yrng = slice(y*wid, (y+1)*wid, factor)

        subdat = visdat[yrng, xrng]
    else:
        wid = 256//(2**z)
        xrng = slice(x*wid, (x+1)*wid)
        yrng = slice(y*wid, (y+1)*wid)
        subdat = visdat[yrng, xrng]
        if z > 1:
            subdat = subdat.repeat(z, 0).repeat(z, 1)
            wid = subdat.shape[0]
              
    if subdat.shape != (wid, wid):
        if 0 in subdat.shape:
            1/0
        else:
            #pad out with nans
            temp = subdat
            subdat = np.empty((256, 256), dtype=temp.dtype)
            subdat.fill(np.nan)
            subdat[:temp.shape[0], :temp.shape[1]] = temp
    
    return ccd_to_pngstr_app(subdat)

**TODO: Do users need to worry about server warning from `th.start()`?**

In [ ]:
th = threading.Thread(target=lambda:app.run(debug=False, use_reloader=False, port=5013))
th.start()

Create an empty `ipyleaflet` viewer. We will display here later.

**TODO: It looks like ipyleaflet is solely to support Earth map view, so using it like this is too hacky?**

In [ ]:
url_templ = 'http://127.0.0.1:5013/fitsNUM/{z}/{x}/{y}.png'

m = ipyleaflet.Map(center=(0, 0), zoom=1, layers=[], 
                   min_zoom=1, scroll_wheel_zoom=True)

# https://github.com/jupyter-widgets/ipyleaflet/issues/332
lbl = ipywidgets.Label()
display(lbl)

# For debugging
lbl2 = ipywidgets.Label()
display(lbl2)


# Event names taken from
# https://github.com/jupyter-widgets/ipyleaflet/blob/5f27207ac7a3f2a08f45181613e9ed9ab37eb759/ipyleaflet/leaflet.py#L100
def handle_interaction(**kwargs):
    global lbl, lbl2
    event_type = kwargs.get('type')
    coo = kwargs.get('coordinates')
    
    if event_type == 'mousemove':
        # TODO: Translate Map coordinates to something meaningful.
        lbl.value = str(coo)
        
    elif event_type == 'click':
        # TODO: How to translate coo to iy and ix?
        lbl2.value = str(kwargs)  #'Mouse is clicked'
        iy = ix = 50
        
        # Matplotlib for 1D
        # TODO: How to re-use the same plot area for subsequent clicks?
        fig, ax = plt.subplots()
        ax.plot(cube[:, iy, ix])


# TODO: Why random int here?
cachebuster_int = random.randint(0, 1000000)
local_fits_layer = ipyleaflet.basemap_to_tiles(
    {'url': url_templ.replace('NUM', str(cachebuster_int)), 
     'attribution': 'fitsfile'})
local_fits_layer.cachebuster_int = cachebuster_int

m.add_layer(local_fits_layer)
m.on_interaction(handle_interaction)

m

# Interactivity:

**TODO: Functions need documentation.**

In [ ]:
def refresh():
    local_fits_layer.cachebuster_int += 1
    local_fits_layer.url = url_templ.replace('NUM', str(local_fits_layer.cachebuster_int))
    
    
def rere_stretch(stretch, i_slice):
    re_stretch(stretch, i_slice)
    refresh()

The following hooks up the slider to display chosen slice in the cube on the viewer above.

In [ ]:
fav_stretch = visualization.LogStretch() + visualization.PercentileInterval(95)


def change_slice(x):
    rere_stretch(fav_stretch, x)

In [ ]:
ipywidgets.interact(change_slice, x=ipywidgets.IntSlider(min=0, max=cube.shape[0]-1, step=1, value=0))